## Load Libraries

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import model_selection, linear_model, tree
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
import warnings; warnings.simplefilter('ignore')
%matplotlib inline
sns.set()
seed = 1234
inputDataset="5dd39447c54ff018a8abbc1a" # Recommendations Input Dataset

INFO:matplotlib.font_manager:generated new fontManager


## Load Data

In [2]:
from data_access_sdk_python.reader import DataSetReader
from datetime import date
reader = DataSetReader()
df = reader.load(data_set_id=inputDataset, ims_org="91D55B255640B3AE7F000101@AdobeOrg")


INFO:DataAccessSDK:There are 1 file(s) in this dataset/batch, limit_files: all
INFO:azure.datalake.store.core:closing stream
INFO:DataAccessSDK:#1/1 batch file platform/5dd39447c54ff018a8abbc1a/datasetViewId=5dd39447c54ff018a8abbc1b/_ACP_YEAR=2019/_ACP_MONTH=11/_ACP_DAY=20/batchId=b26690c0-0b8a-11ea-b5ba-799aacf714bd/part-00000-tid-3988023138967601034-94251a45-df57-458f-9533-8fbfb9b04a7a-22464487.c000.snappy.parquet : 16 rows read. 219.09 MB memory used for this process.
INFO:DataAccessSDK:16 rows total in dataframe


## Summarize the Data

### Dataset Dimensions

In [3]:
print("You'll see the number of lines and the number of columns:", df.shape)

You'll see the number of lines and the number of columns: (16, 5)


### Peeking at the Data

In [4]:
df.head(20)

,timestamp,_ibmnaamericaspartnersandbox.brandName,_ibmnaamericaspartnersandbox.interactionType,_ibmnaamericaspartnersandbox.itemId,_ibmnaamericaspartnersandbox.userId
0,2019-11-20 08:44:37,Luma Retail,productView,Nadia Elements Shell,91635799509879697768509199328383330031
1,2019-11-20 08:44:44,Luma Retail,productView,Nadia Elements Shell,91635799509879697768509199328383330031
2,2019-11-20 08:44:59,Luma Retail,productView,Nadia Elements Shell,91635799509879697768509199328383330031
3,2019-11-20 08:44:59,Luma Retail,productView,Nadia Elements Shell,91635799509879697768509199328383330031
4,2019-11-20 08:45:31,Luma Retail,productView,Nadia Elements Shell,91635799509879697768509199328383330031
5,2019-11-20 08:45:39,Luma Retail,productView,Nadia Elements Shell,91635799509879697768509199328383330031
6,2019-11-20 08:47:45,Luma Retail,productView,Nadia Elements Shell,91635799509879697768509199328383330031
7,2019-11-20 08:47:45,Luma Retail,productView,Nadia Elements Shell,91635799509879697768509199328383330031
8,2019-11-20 08:48:09,Luma Retail,productView,Nadia Elements Shell,91635799509879697768509199328383330031
9,2019-11-20 08:48:09,Luma Retail,productView,Nadia Elements Shell,91635799509879697768509199328383330031


### Statistical Summary

In [6]:
user_id = '_ibmnaamericaspartnersandbox.userId'
item_id = '_ibmnaamericaspartnersandbox.itemId'
interaction_type = '_ibmnaamericaspartnersandbox.interactionType'
timestamp = '_ibmnaamericaspartnersandbox.timestamp'

In [7]:
df.describe()

,timestamp,_ibmnaamericaspartnersandbox.brandName,_ibmnaamericaspartnersandbox.interactionType,_ibmnaamericaspartnersandbox.itemId,_ibmnaamericaspartnersandbox.userId
count,16,16,16,16,16
unique,11,1,1,1,1
top,2019-11-20 08:48:09,Luma Retail,productView,Nadia Elements Shell,91635799509879697768509199328383330031
freq,2,16,16,16,16
first,2019-11-20 08:44:37,NaN,NaN,NaN,NaN
last,2019-11-20 08:52:13,NaN,NaN,NaN,NaN


### Grouping Items by Item ID

In [8]:
df[[user_id, item_id]].groupby([item_id]).agg(['count'])

,_ibmnaamericaspartnersandbox.userId
,count
_ibmnaamericaspartnersandbox.itemId,
Nadia Elements Shell,16
